In [1]:
import pandas as pd 
import numpy as np
import math
from itertools import combinations

df = pd.read_csv('https://raw.githubusercontent.com/AugustLONG/ML01/master/01decisiontree/AllElectronics.csv')
df.drop("RID",axis=1, inplace = True) #RID는 그냥 Index라서 삭제

In [2]:
df

,age,income,student,credit_rating,class_buys_computer
0,youth,high,no,fair,no
1,youth,high,no,excellent,no
2,middle_aged,high,no,fair,yes
3,senior,medium,no,fair,yes
4,senior,low,yes,fair,yes
5,senior,low,yes,excellent,no
6,middle_aged,low,yes,excellent,yes
7,youth,medium,no,fair,no
8,youth,low,yes,fair,yes
9,senior,medium,yes,fair,yes


## 함수 만들기

In [3]:
def getEntropy(df, feature):
    yes_num = 0
    no_num = 0
    for data in df[feature]:
        if data == 'yes':
            yes_num += 1
        else:
            no_num += 1
            
    # 이 밑은 log 값에 0이들어가면 에러가 나서 예외처리를 해주었습니다.
    
    if yes_num == 0:
        entropy = - (no_num / (yes_num + no_num) * math.log(no_num / (yes_num + no_num), 2))
    elif no_num == 0:
        entropy = - (yes_num / (yes_num + no_num) * math.log(yes_num / (yes_num + no_num), 2))
    else:
        entropy = - (yes_num / (yes_num + no_num) * math.log(yes_num / (yes_num + no_num), 2) + no_num / (yes_num + no_num) * math.log(no_num / (yes_num + no_num), 2))
    
    return entropy

In [4]:
getEntropy(df, "class_buys_computer")

0.9402859586706309

In [5]:
def getGainA(df, feature) :
    info_D = getEntropy(df, feature) # 목표변수 Feature에 대한 Info(Entropy)를 구한다.
    columns = list(df.loc[:, df.columns != feature]) # 목표변수를 제외한 나머지 설명변수들을 리스트 형태로 저장한다.

    result = {}

    for context in columns:
        Info_context = [] # 인포값 담을 리스트
        col_list = [] # 칼럼이 가지는 값의 종류
        context_df = df[[context, feature]] # 필요변수만 가지고 새로운 데이터프레임 생성
        kkk = set(list(combinations(df[context], 1)))

        for i in kkk:
            col_list.append((list(i)[0]))  # 리스트 형변환

        for i in col_list:
            new_context_df = context_df[context_df[context] == i] # 엔트로피 계산을 위하여 다시 데이터프레임 분리
            Info_context.append(getEntropy(new_context_df, "class_buys_computer") / len(col_list))
            # 인포 계산 후 결과값에 담기!
        result[context] = info_D - sum(Info_context)

    return result


In [6]:
print(getGainA(df, "class_buys_computer"))

{'age': 0.2929855623675185, 'income': 0.030427972499423417, 'student': 0.15183550136234136, 'credit_rating': 0.034646896441064445}


## 결과 확인하기

In [7]:
my_dict = getGainA(df, "class_buys_computer")
def f1(x):
    return my_dict[x]
key_max = max(my_dict.keys(), key=f1)
print('정보 획득이 가장 높은 변수는',key_max, "이며 정보 획득량은", my_dict[key_max], "이다.")

정보 획득이 가장 높은 변수는 age 이며 정보 획득량은 0.2929855623675185 이다.
